---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

### **`캐시 임베딩 (CacheBackedEmbeddings)`**

### **1) `CacheBackedEmbeddings`**

* **`Embeddings` = 재계산을 피하기 위해 저장되거나 `일시적으로 캐시`될 수 있음**

  * `Embeddings`를 캐싱하는 것 = `CacheBackedEmbeddings` 사용 → 수행

  * `캐시 지원 embedder` = `embeddings`를 **`키-값 저장소`에 `캐싱`하는 `embedder` `주변 래퍼`** 
    * `텍스트 = 해시` → `캐시`에서 `키`로 `사용`

---

* 기초 개념

  * 한 번 계산된 임베딩 벡터를 `메모리`나 `파일`에 `저장`해두고 `재사용`하는 기술
  * **`같은 텍스트`에 대해서는 `API`를 다시 호출하지 않고 `저장된 결과 사용`**
  * 개발자의 시간과 비용을 크게 절약하는 필수 최적화 기법

* 필요성

  *  **`비용 절약`**

    ```python

      # 일반 임베딩: 매번 API 호출
      texts = ["안녕하세요", "안녕하세요", "안녕하세요"]          # 같은 텍스트 3번
      for text in texts:
      embedding = openai_embed(text)                      # 3번 API 호출 = $0.03

      # 캐시백드 임베딩: 첫 번째만 API 호출
      cache = {}
      for text in texts:
      if text in cache:
      embedding = cache[text]                             # 캐시에서 가져오기
      else:
      embedding = openai_embed(text)                      # 1번만 API 호출 = $0.01
      cache[text] = embedding                             # 약 66% 비용 절약 (같은 텍스트가 많을수록 더 크게 절약됨)

    ```

  *  **`속도 향상`**

    ```markdown

      | 구분      | API 호출       | 캐시 조회     | 속도 차이          |
      |----------|---------------|-------------|-----------------|
      | 일반 임베딩 | 2-3초          | -          | 기준              |
      | 캐시백드    | 2-3초 (첫 번째) | 0.01-0.1초  | **20-300배 빠름** |
    
    ```

  *  **`개발 효율성`**

    ```python
    
      # 개발/테스트 시나리오
      for i in range(10): # 같은 데이터로 10번 테스트
        # 일반: 매번 25초 = 총 250초 (4분 10초)
        # 캐시: 첫 번째 25초 + 나머지 9초 = 총 34초
        print(f"테스트 {i+1} 완료")
        
        # 개발 효율성 85% 향상!

    ```

---

* **`CacheBackedEmbeddings`를 초기화하는 주요 지원 방법 = `from_bytes_store`**

* **`매개변수`**:

  * `underlying_embeddings` = `임베딩`을 위해 사용되는 `embedder`

  * `document_embedding_cache` = `문서 임베딩`을 `캐싱`하기 위한 `ByteStore` 중 하나

  * **`namespace` (선택 사항, 기본값은 `""`)**

    * 문서 캐시를 위해 사용되는 `네임스페이스`

    * `다른 캐시와의 충돌을 피하기 위해 사용`
      * 예시: 사용된 임베딩 모델의 이름으로 설정

* *주의*: 동일한 텍스트가 다른 임베딩 모델을 사용하여 임베딩될 때 **`충돌을 피하기 위해 namespace 매개변수를 설정하는 것이 중요`**

---

#### **❌ 1) `LocalFileStore`에서 `임베딩` 사용** (영구 보관)

* **`로컬 파일` 시스템 사용 → `임베딩 저장` → `FAISS 벡터 스토어`를 `사용`하여 `검색`하는 예제**

* **`gemini-embedding`모델 사용해보기**

  * `gemini-embedding` 사용 시 **`task-type` 명시해야 함**

  * *`task-type` 종류*

    * **`retrieval_document`**: `문서 검색`을 위한 임베딩 생성

    * **`retrieval_query`**: `쿼리 검색`을 위한 임베딩 생성

    * **`semantic_similarity`**: `의미적 유사성`을 위한 임베딩 생성

    * **`classification`**: `분류`를 위한 임베딩 생성

    * **`clustering`**: `클러스터링`을 위한 임베딩 생성

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
import os

load_dotenv()

# API 키 확인
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = input("Enter your Google API key: ")

# Gemini 임베딩 모델 생성 (task_type 명시)
embeddings = GoogleGenerativeAIEmbeddings(
    #model="models/gemini-embedding-001",
    model="gemini-embedding-001",
    task_type="retrieval_document",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

<small>

* 셀 출력

    ```markdown
    WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
    E0000 00:00:1758420024.623257 1615563 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
    ```

<small>

* 셀 출력

  ```bash
    /Users/jay/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
      _warn_about_sha1_encoder()
  ```

* 오류 메시지
  * 의미: `기본 키 인코더인 SHA-1은 충돌에 취약합니다. 대부분의 캐시 시나리오에서는 허용되지만, 의도한 공격자가 동일한 캐시 키로 매핑되는 두 가지 다른 페이로드를 만들 수 있습니다.`
  * 원인: `CacheBackedEmbeddings` 클래스 = 기본적으로 `SHA-1 키 인코더`를 사용하기 때문
  * 해결방법: 
    * `key_encoder` 매개변수를 사용하여 더 강력한 인코더(예: SHA-256 또는 BLAKE2)를 제공
    * 키 인코더를 변경한 경우, 기존 키와의 충돌을 피하기 위해 새로운 캐시를 생성하는 것을 고려

In [ ]:
# 오류 생긴 코드 셀

from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import hashlib

# 기본 임베딩 설정 (gemini-embedding 모델)
embedding = embeddings

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")                                  # 현재 작업 디렉토리의 하위 디렉토리로 생성

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,                                      # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
    key_encoder=lambda x: hashlib.sha256(x.encode()).hexdigest()    # SHA-256 키 인코더 사용
)

<small>

* 오류 O

* 오류 메시지
```bash

    ValueError: Do not supply namespace when using a custom key_encoder; add any prefixing inside the encoder itself.

```

  * 해석: `사용자 정의 key_encoder를 사용할 때 namespace를 제공하지 마세요. 접두사를 인코더 자체에 추가하세요.`
  * 원인: `key_encoder`를 사용자 정의할 때 `namespace`를 함께 제공하면 충돌 발생 가능
  * 해결 방법: `namespace`를 제거하고 `key_encoder`에 접두사 추가하기

In [3]:
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import hashlib

# 기본 임베딩 설정 (gemini-embedding 모델)
embedding = embeddings

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")                                  # 현재 작업 디렉토리의 하위 디렉토리로 생성

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    key_encoder=lambda x: f"{embedding.model}:{hashlib.sha256(x.encode()).hexdigest()}"    # SHA-256 키 인코더 사용 및 접두사 추가
)

In [ ]:
print(type(cached_embedder))        # <class 'langchain.embeddings.cache.CacheBackedEmbeddings'>

In [ ]:
# store에서 키들을 순차적으로 가져오기

list(store.yield_keys())

<small>

* 셀 출력

    ```python
    []
    ```

* **문서 로드 → 청크 분할 → 청크 임베딩 → 벡터 저장소에 로드**

In [5]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("../08_Embedding/data/appendix-keywords.txt").load()

# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0)

# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [ ]:
print(type(documents))      # <class 'list'>

In [ ]:
print(documents[0].page_content)

<small>

* 셀 출력

    ```markdown
    Semantic Search

    정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
    예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
    연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

    Embedding

    정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
    예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
    연관키워드: 자연어 처리, 벡터화, 딥러닝

    Token

    정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
    예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
    연관키워드: 토큰화, 자연어 처리, 구문 분석

    Tokenizer

    정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
    예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
    연관키워드: 토큰화, 자연어 처리, 구문 분석

    VectorStore

    정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
    예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
    연관키워드: 임베딩, 데이터베이스, 벡터화

    SQL
    ```

In [ ]:
print(documents[0].metadata)

<small>

* 셀 출력

    ```python
        {'source': '../08_Embedding/data/appendix-keywords.txt'}
    ```

In [10]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore
import hashlib

# 문서 임베딩 생성
document_embeddings = cached_embedder.embed_documents

In [11]:
# 쿼리 임베딩 생성
query_embedding = cached_embedder.embed_query

In [ ]:
print(type(document_embeddings))            # <class 'method'>
print(type(query_embedding))                # <class 'method'>

In [ ]:
# 결과 출력
print("문서 임베딩:")

for i, doc_embedding in enumerate(document_embeddings):
    print(f"문서 {i+1} 임베딩 길이: {len(doc_embedding)}")

print("\n쿼리 임베딩 길이:", len(query_embedding))

---

<small>

* 오류 메시지

    ```markdown
    문서 임베딩:
    ```

    ```bash
    ---------------------------------------------------------------------------
    TypeError                                 Traceback (most recent call last)
    Cell In[12], line 3
        1 # 결과 출력
        2 print("문서 임베딩:")
    ----> 3 for i, doc_embedding in enumerate(document_embeddings):
        4     print(f"문서 {i+1} 임베딩 길이: {len(doc_embedding)}")
        6 print("\n쿼리 임베딩 길이:", len(query_embedding))

    TypeError: 'method' object is not iterable
    ```

---

* 사전에 `VS Code` 터미널에 설치할 것 

```bash
        pip install faiss-cpu
```

In [ ]:
# 코드 실행 시간을 측정하기
# 문서로부터 FAISS 데이터베이스 생성
from langchain_community.vectorstores import FAISS

%time db = FAISS.from_documents(documents, cached_embedder)

<small>

* 셀 출력

    ```markdown
    CPU times: user 143 μs, sys: 11 μs, total: 154 μs
    Wall time: 155 μs
    ```

    ```bash
    ---------------------------------------------------------------------------
    InvalidKeyException                       Traceback (most recent call last)
    Cell In[5], line 5
        1 # 코드 실행 시간을 측정하기
        2 # 문서로부터 FAISS 데이터베이스 생성
        3 from langchain_community.vectorstores import FAISS
    ----> 5 get_ipython().run_line_magic('time', 'db = FAISS.from_documents(documents, cached_embedder)')

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:2504, in InteractiveShell.run_line_magic(self, magic_name, line, _stack_depth)
    2502     kwargs['local_ns'] = self.get_local_scope(stack_depth)
    2503 with self.builtin_trap:
    -> 2504     result = fn(*args, **kwargs)
    2506 # The code below prevents the output from being displayed
    2507 # when using magics with decorator @output_can_be_silenced
    2508 # when the last Python token in the expression is a ';'.
    2509 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/magics/execution.py:1470, in ExecutionMagics.time(self, line, cell, local_ns)
    1468 if interrupt_occured:
    1469     if exit_on_interrupt and captured_exception:
    -> 1470         raise captured_exception
    1471     return
    1472 return out

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/magics/execution.py:1434, in ExecutionMagics.time(self, line, cell, local_ns)
    1432 st = clock2()
    1433 try:
    -> 1434     exec(code, glob, local_ns)
    1435     out = None
    1436     # multi-line %%time case

    File <timed exec>:1

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/vectorstores/base.py:837, in VectorStore.from_documents(cls, documents, embedding, **kwargs)
        834     if any(ids):
        835         kwargs["ids"] = ids
    --> 837 return cls.from_texts(texts, embedding, metadatas=metadatas, **kwargs)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_community/vectorstores/faiss.py:1043, in FAISS.from_texts(cls, texts, embedding, metadatas, ids, **kwargs)
    1016 @classmethod
    1017 def from_texts(
    1018     cls,
    (...)   1023     **kwargs: Any,
    1024 ) -> FAISS:
    1025     """Construct FAISS wrapper from raw documents.
    1026 
    1027     This is a user friendly interface that:
    (...)   1041             faiss = FAISS.from_texts(texts, embeddings)
    1042     """
    -> 1043     embeddings = embedding.embed_documents(texts)
    1044     return cls.__from(
    1045         texts,
    1046         embeddings,
    (...)   1050         **kwargs,
    1051     )

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/embeddings/cache.py:178, in CacheBackedEmbeddings.embed_documents(self, texts)
        165 def embed_documents(self, texts: list[str]) -> list[list[float]]:
        166     """Embed a list of texts.
        167 
        168     The method first checks the cache for the embeddings.
    (...)    176         A list of embeddings for the given texts.
        177     """
    --> 178     vectors: list[Union[list[float], None]] = self.document_embedding_store.mget(
        179         texts,
        180     )
        181     all_missing_indices: list[int] = [
        182         i for i, vector in enumerate(vectors) if vector is None
        183     ]
        185     for missing_indices in batch_iterate(self.batch_size, all_missing_indices):

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/storage/encoder_backed.py:67, in EncoderBackedStore.mget(self, keys)
        65 """Get the values associated with the given keys."""
        66 encoded_keys: list[str] = [self.key_encoder(key) for key in keys]
    ---> 67 values = self.store.mget(encoded_keys)
        68 return [
        69     self.value_deserializer(value) if value is not None else value
        70     for value in values
        71 ]

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/storage/file_system.py:122, in LocalFileStore.mget(self, keys)
        120 values: list[Optional[bytes]] = []
        121 for key in keys:
    --> 122     full_path = self._get_full_path(key)
        123     if full_path.exists():
        124         value = full_path.read_bytes()

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/storage/file_system.py:79, in LocalFileStore._get_full_path(self, key)
        77 if not re.match(r"^[a-zA-Z0-9_.\-/]+$", key):
        78     msg = f"Invalid characters in key: {key}"
    ---> 79     raise InvalidKeyException(msg)
        80 full_path = (self.root_path / key).resolve()
        81 root_path = self.root_path.resolve()

    InvalidKeyException: Invalid characters in key: gemini-embedding-001:a4b7261126432c208767fc2ffc0033a6839d8cb986329af5efc0580a17e2e047
    ```

* 벡터 저장소를 다시 생성하려고 하면, 임베딩을 다시 계산할 필요가 없기 때문에 훨씬 더 빠르게 처리됨

In [ ]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성

%time db2 = FAISS.from_documents(documents, cached_embedder)

<small>

* 셀 출력

    ```markdown
    CPU times: user 322 μs, sys: 666 μs, total: 988 μs
    Wall time: 813 μs
    ```

    ```bash
    ---------------------------------------------------------------------------
    InvalidKeyException                       Traceback (most recent call last)
    Cell In[6], line 3
        1 # 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
    ----> 3 get_ipython().run_line_magic('time', 'db2 = FAISS.from_documents(documents, cached_embedder)')

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:2504, in InteractiveShell.run_line_magic(self, magic_name, line, _stack_depth)
    2502     kwargs['local_ns'] = self.get_local_scope(stack_depth)
    2503 with self.builtin_trap:
    -> 2504     result = fn(*args, **kwargs)
    2506 # The code below prevents the output from being displayed
    2507 # when using magics with decorator @output_can_be_silenced
    2508 # when the last Python token in the expression is a ';'.
    2509 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/magics/execution.py:1470, in ExecutionMagics.time(self, line, cell, local_ns)
    1468 if interrupt_occured:
    1469     if exit_on_interrupt and captured_exception:
    -> 1470         raise captured_exception
    1471     return
    1472 return out

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/magics/execution.py:1434, in ExecutionMagics.time(self, line, cell, local_ns)
    1432 st = clock2()
    1433 try:
    -> 1434     exec(code, glob, local_ns)
    1435     out = None
    1436     # multi-line %%time case

    File <timed exec>:1

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_core/vectorstores/base.py:837, in VectorStore.from_documents(cls, documents, embedding, **kwargs)
        834     if any(ids):
        835         kwargs["ids"] = ids
    --> 837 return cls.from_texts(texts, embedding, metadatas=metadatas, **kwargs)

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_community/vectorstores/faiss.py:1043, in FAISS.from_texts(cls, texts, embedding, metadatas, ids, **kwargs)
    1016 @classmethod
    1017 def from_texts(
    1018     cls,
    (...)   1023     **kwargs: Any,
    1024 ) -> FAISS:
    1025     """Construct FAISS wrapper from raw documents.
    1026 
    1027     This is a user friendly interface that:
    (...)   1041             faiss = FAISS.from_texts(texts, embeddings)
    1042     """
    -> 1043     embeddings = embedding.embed_documents(texts)
    1044     return cls.__from(
    1045         texts,
    1046         embeddings,
    (...)   1050         **kwargs,
    1051     )

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/embeddings/cache.py:178, in CacheBackedEmbeddings.embed_documents(self, texts)
        165 def embed_documents(self, texts: list[str]) -> list[list[float]]:
        166     """Embed a list of texts.
        167 
        168     The method first checks the cache for the embeddings.
    (...)    176         A list of embeddings for the given texts.
        177     """
    --> 178     vectors: list[Union[list[float], None]] = self.document_embedding_store.mget(
        179         texts,
        180     )
        181     all_missing_indices: list[int] = [
        182         i for i, vector in enumerate(vectors) if vector is None
        183     ]
        185     for missing_indices in batch_iterate(self.batch_size, all_missing_indices):

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/storage/encoder_backed.py:67, in EncoderBackedStore.mget(self, keys)
        65 """Get the values associated with the given keys."""
        66 encoded_keys: list[str] = [self.key_encoder(key) for key in keys]
    ---> 67 values = self.store.mget(encoded_keys)
        68 return [
        69     self.value_deserializer(value) if value is not None else value
        70     for value in values
        71 ]

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/storage/file_system.py:122, in LocalFileStore.mget(self, keys)
        120 values: list[Optional[bytes]] = []
        121 for key in keys:
    --> 122     full_path = self._get_full_path(key)
        123     if full_path.exists():
        124         value = full_path.read_bytes()

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/storage/file_system.py:79, in LocalFileStore._get_full_path(self, key)
        77 if not re.match(r"^[a-zA-Z0-9_.\-/]+$", key):
        78     msg = f"Invalid characters in key: {key}"
    ---> 79     raise InvalidKeyException(msg)
        80 full_path = (self.root_path / key).resolve()
        81 root_path = self.root_path.resolve()

    InvalidKeyException: Invalid characters in key: gemini-embedding-001:a4b7261126432c208767fc2ffc0033a6839d8cb986329af5efc0580a17e2e047
    ```

---

#### **❌ 2) `InmemoryByteStore` 사용** (비영구적)

* **`ByteStore` 사용: `CacheBackedEmbeddings` 생성할 때 `해당 ByteStore를 사용`**

* 비영구적인 **`InmemoryByStore` 사용** → 동일한 캐시된 임베딩 객체를 생성하는 예시

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

# 메모리 내 바이트 저장소 생성
store = InMemoryByteStore()  

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model
)

<small>

* 오류 메시지

  ```bash
  /Users/jay/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
    _warn_about_sha1_encoder()
  ```

* 오류 메시지 해석, 원인, 해결 방법
  * 해석: 기본 키 인코더인 SHA-1은 충돌에 취약합니다. 대부분의 캐시 시나리오에서는 허용되지만, 의도한 공격자가 동일한 캐시 키로 매핑되는 두 가지 다른 페이로드를 만들 수 있습니다.
  * 원인: CacheBackedEmbeddings 클래스가 기본적으로 SHA-1 키 인코더를 사용하기 때문입니다.
  * 해결 방법: key_encoder 매개변수를 사용하여 더 강력한 인코더(예: SHA-256 또는 BLAKE2)를 제공합니다. 키 인코더를 변경한 경우, 기존 키와의 충돌을 피하기 위해 새로운 캐시를 생성하는 것을 고려

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore
import hashlib

# 메모리 내 바이트 저장소 생성
store = InMemoryByteStore()

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model,
    key_encoder=lambda x: hashlib.sha256(x.encode()).hexdigest()  # SHA-256 키 인코더 사용
)

<small>

* 오류 메시지

```bash
    ValueError: Do not supply namespace when using a custom key_encoder; add any prefixing inside the encoder itself.
```

* 오류 메시지 해석, 원인, 해결 방법
  * 해석: 사용자 정의 key_encoder를 사용할 때 namespace를 제공하지 마세요. 접두사를 인코더 자체에 추가하세요.
  * 원인: key_encoder를 사용자 정의할 때 namespace를 함께 제공하면 충돌이 발생할 수 있습니다.
  * 해결 방법: namespace를 제거하고 key_encoder에 접두사를 추가

In [9]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore
import hashlib

# 메모리 내 바이트 저장소 생성
store = InMemoryByteStore()

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store,
    key_encoder=lambda x: f"{embedding.model}:{hashlib.sha256(x.encode()).hexdigest()}"  # SHA-256 키 인코더 사용 및 접두사 추가
)

In [15]:
# 문서 로드, 청크 분할, 임베딩, 벡터 저장소에 로드

from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("../08_Embedding/data/appendix-keywords.txt").load()

# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [ ]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성

%time db3 = FAISS.from_documents(documents, cached_embedder)

<small>

* 셀 출력

    ```markdown
    CPU times: user 9 μs, sys: 1 μs, total: 10 μs
    Wall time: 18.1 μs
    ```

    ```bash
    ---------------------------------------------------------------------------
    NameError                                 Traceback (most recent call last)
    Cell In[16], line 3
        1 # 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
    ----> 3 get_ipython().run_line_magic('time', 'db3 = FAISS.from_documents(documents, cached_embedder)')

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/interactiveshell.py:2504, in InteractiveShell.run_line_magic(self, magic_name, line, _stack_depth)
    2502     kwargs['local_ns'] = self.get_local_scope(stack_depth)
    2503 with self.builtin_trap:
    -> 2504     result = fn(*args, **kwargs)
    2506 # The code below prevents the output from being displayed
    2507 # when using magics with decorator @output_can_be_silenced
    2508 # when the last Python token in the expression is a ';'.
    2509 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/magics/execution.py:1470, in ExecutionMagics.time(self, line, cell, local_ns)
    1468 if interrupt_occured:
    1469     if exit_on_interrupt and captured_exception:
    -> 1470         raise captured_exception
    1471     return
    1472 return out

    File ~/.pyenv/versions/lc_env/lib/python3.13/site-packages/IPython/core/magics/execution.py:1434, in ExecutionMagics.time(self, line, cell, local_ns)
    1432 st = clock2()
    1433 try:
    -> 1434     exec(code, glob, local_ns)
    1435     out = None
    1436     # multi-line %%time case

    File <timed exec>:1

    NameError: name 'FAISS' is not defined
    ```

In [17]:
# 문서 임베딩 생성
document_embeddings = cached_embedder.embed_documents

In [18]:
# 쿼리 임베딩 생성
query_embedding = cached_embedder.embed_query

In [ ]:
# 결과 출력
print("문서 임베딩:")

for i, doc_embedding in enumerate(document_embeddings):
    print(f"문서 {i+1} 임베딩 길이: {len(doc_embedding)}")

print("\n쿼리 임베딩 길이:", len(query_embedding))

<small>

* 오류 메시지

    ```markdown
    문서 임베딩:
    ```

    ```bash
    ---------------------------------------------------------------------------
    TypeError                                 Traceback (most recent call last)
    Cell In[19], line 4
        1 # 결과 출력
        2 print("문서 임베딩:")
    ----> 4 for i, doc_embedding in enumerate(document_embeddings):
        5     print(f"문서 {i+1} 임베딩 길이: {len(doc_embedding)}")
        7 print("\n쿼리 임베딩 길이:", len(query_embedding))

    TypeError: 'method' object is not iterable
    ```

---

#### **3) `재시도`**

* 기본 설정

In [3]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore, LocalFileStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import hashlib
import time
import os

# 환경 변수 로드
load_dotenv()

# Google API 키 설정
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = input("Enter your Google API key: ")

* `gemini-embedding` 모델 생성

In [8]:
# Gemini 임베딩 모델 설정
base_embeddings = GoogleGenerativeAIEmbeddings(
    #model="models/gemini-embedding-001",
    model="gemini-embedding-001",
    task_type="retrieval_document",                         # 문서 검색용
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# 다른 task_type 옵션들:
# task_type="retrieval_document": 문서 검색을 위한 임베딩 생성
# task_type="retrieval_query": 쿼리 검색을 위한 임베딩 생성  
# task_type="semantic_similarity": 의미적 유사성을 위한 임베딩 생성
# task_type="classification": 분류를 위한 임베딩 생성
# task_type="clustering": 클러스터링을 위한 임베딩 생성

# 테스트용 문서들
documents = [
    "캐시백드 임베딩은 성능 최적화의 핵심입니다.",
    "LangChain을 활용한 AI 개발이 효율적입니다.", 
    "Jay의 AI 교육은 실무 중심으로 진행됩니다.",
    "캐시백드 임베딩은 성능 최적화의 핵심입니다.",                      # 중복된 텍스트
]


E0000 00:00:1758626579.449795 1577171 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


<small>

* 셀 출력

```bash
    WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
    E0000 00:00:1758625382.045711 1173394 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
```

---

* **`캐시 저장소` 선택하기**

*  * `option_1`: **`임시 메모리` 저장**

In [9]:
# 프로그램 실행 중에만 유지(빠름)

memory_store = InMemoryByteStore()

*  * `option_2`: **`영구 파일` 저장**

In [10]:
# 파일로 저장하여 재시작 후에도 유지 (권장)

file_store = LocalFileStore("../08_Embedding/embeddings_cache/")

---

* **`안전한 키 생성` 함수 사용해보기**

In [11]:
def safe_key_encoder(text: str) -> str:
    """
    텍스트를 안전한 캐시 키로 변환
    - SHA256 해시로 고정 길이 키 생성
    - 특수문자나 긴 텍스트 문제 해결
    - Gemini 모델명을 prefix로 추가
    """
    hash_object = hashlib.sha256(text.encode('utf-8'))
    hex_dig = hash_object.hexdigest()
    return f"gemini_embedding_001:{hex_dig}"

# 테스트
print(safe_key_encoder("안녕하세요"))

gemini_embedding_001:2c68318e352971113645cbc72861e1ec23f48d5baa5f9b405fed9dddca893eb4


<small>

* 셀 출력 

    ```bash
        gemini_embedding_001:2c68318e352971113645cbc72861e1ec23f48d5baa5f9b405fed9dddca893eb4
    ```

---

* **`캐시백드 임베딩` 생성**

In [12]:
# 캐시백드 임베딩 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=base_embeddings,                          # gemini 임베딩 모델
    document_embedding_cache=file_store,                            # 캐시 저장소
    key_encoder=safe_key_encoder                                    # 키 생성 함수 (namespace 대신 사용)
)

print("✅ Gemini 캐시백드 임베딩 생성 완료!")                             # ✅ Gemini 캐시백드 임베딩 생성 완료!

✅ Gemini 캐시백드 임베딩 생성 완료!


---

* **`성능 비교 테스트`**

*  * **`실제 성능 측정 코드`**

In [ ]:
import time
from typing import List

def performance_test():
    """일반 Gemini 임베딩 vs 캐시백드 임베딩 성능 비교"""
    
    test_texts = [
        "캐시백드 임베딩 테스트",
        "LangChain 성능 최적화", 
        "Jay의 AI 교육 프로그램",
        "캐시백드 임베딩 테스트",                              # 중복된 텍스트
        "LangChain 성능 최적화",                            # 중복된 텍스트
    ]
    
    # 일반 Gemini 임베딩 테스트
    print("🐌 일반 Gemini 임베딩 테스트 시작...")
    start_time = time.time()
    
    regular_embeddings = []
    for i, text in enumerate(test_texts):
        print(f"  텍스트 {i+1} 처리 중...")
        embedding = base_embeddings.embed_query(text)
        regular_embeddings.append(embedding)
    
    regular_time = time.time() - start_time
    print(f"일반 Gemini 임베딩 소요시간: {regular_time:.2f}초")
    
    # 캐시백드 임베딩 테스트  
    print("\n⚡ 캐시백드 Gemini 임베딩 테스트 시작...")
    start_time = time.time()
    
    cached_results = []
    for i, text in enumerate(test_texts):
        print(f"  텍스트 {i+1} 처리 중...")
        embedding = cached_embeddings.embed_query(text)
        cached_results.append(embedding)
    
    cached_time = time.time() - start_time
    print(f"캐시백드 임베딩 소요시간: {cached_time:.2f}초")
    
    # 결과 분석
    speedup = regular_time / cached_time if cached_time > 0 else float('inf')
    print(f"\n📊 성능 개선 결과:")
    print(f"   속도 향상: {speedup:.1f}배")
    print(f"   시간 절약: {regular_time - cached_time:.2f}초")
    
# 테스트 실행
performance_test()

<small>

* 셀 출력 (5.3s)

    ```markdown
    🐌 일반 Gemini 임베딩 테스트 시작...
    텍스트 1 처리 중...
    텍스트 2 처리 중...
    텍스트 3 처리 중...
    텍스트 4 처리 중...
    텍스트 5 처리 중...
    일반 Gemini 임베딩 소요시간: 2.50초

    ⚡ 캐시백드 Gemini 임베딩 테스트 시작...
    텍스트 1 처리 중...
    텍스트 2 처리 중...
    텍스트 3 처리 중...
    텍스트 4 처리 중...
    텍스트 5 처리 중...
    캐시백드 임베딩 소요시간: 2.80초

    📊 성능 개선 결과:
    속도 향상: 0.9배
    시간 절약: -0.29초
    ```

---

* **`벡터 스토어` 함께 사용**

In [ ]:
# 기존 캐시 폴더 정리 (새로 시작)
import shutil
import os

cache_dir = "./embeddings_cache/"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("🧹 기존 캐시 폴더 삭제 완료")            # 🧹 기존 캐시 폴더 삭제 완료

In [3]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import hashlib
import time

# 환경 변수 로드
load_dotenv()

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = input("Enter your Google API key: ")

# 캐시 폴더 정리
cache_dir = "../08_Embedding/embeddings_cache/"         # "./embeddings_cache/"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

file_store = LocalFileStore(cache_dir)

In [ ]:
# 올바른 모델명으로 Gemini 임베딩 설정

base_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",                # 🔥 models/ 접두사 추가!
    task_type="retrieval_document",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

<small>

* 셀 출력

    ```bash
        E0000 00:00:1758627061.937791 1577171 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
    ```

In [ ]:
# 모델명 테스트
def test_model_name():
    try:
        # 간단한 임베딩 테스트
        test_embedding = base_embeddings.embed_query("테스트")
        print(f"✅ 모델명 테스트 성공! 임베딩 길이: {len(test_embedding)}")
        return True
    except Exception as e:
        print(f"❌ 모델명 테스트 실패: {e}")
        return False

# 테스트 실행
test_model_name()                           # ✅ 모델명 테스트 성공! 임베딩 길이: 3072, True

In [ ]:
# 안전한 키 생성 함수
def safe_key_encoder(text: str) -> str:
    hash_object = hashlib.sha256(text.encode('utf-8'))
    hex_dig = hash_object.hexdigest()
    return f"gemini_embedding_001_{hex_dig}"            # 언더스코어 사용


# 캐시백드 임베딩 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=base_embeddings,
    document_embedding_cache=file_store,
    key_encoder=safe_key_encoder
)

print("✅ 올바른 모델명으로 캐시백드 임베딩 생성 완료!")           # ✅ 올바른 모델명으로 캐시백드 임베딩 생성 완료!

In [ ]:
# 벡터스토어 생성 함수
def create_vector_store_with_cache_final():
    documents = [
        "캐시백드 임베딩으로 RAG 시스템 최적화",
        "Jay의 프롬프트 엔지니어링 강의",
        "LangChain 실무 프로젝트 가이드",
        "AI 교육 사업의 성공 전략",
        "캐시백드 임베딩으로 RAG 시스템 최적화",                # 중복된 텍스트
    ]
    
    print("📚 올바른 모델명으로 Gemini 벡터스토어 생성 중...")
    start_time = time.time()
    
    try:
        vector_store = FAISS.from_texts(
            texts=documents,
            embedding=cached_embeddings                 # 수정된 캐시백드 임베딩
        )
        
        creation_time = time.time() - start_time
        print(f"✅ 벡터스토어 생성 완료: {creation_time:.2f}초")
        
        # 검색 테스트
        query = "AI 교육"
        results = vector_store.similarity_search(query, k=2)
        
        print(f"\n🔍 검색 결과 ('{query}'):")
        for i, doc in enumerate(results):
            print(f"  {i+1}. {doc.page_content}")
        
        return vector_store
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

# 실행
vector_store = create_vector_store_with_cache_final()

<small>

* 셀 출력

    ```markdown
    📚 올바른 모델명으로 Gemini 벡터스토어 생성 중...
    ✅ 벡터스토어 생성 완료: 0.88초

    현재 셀 또는 이전 셀에서 코드를 실행하는 동안 Kernel이 충돌했습니다. 
    셀의 코드를 검토하여 가능한 오류 원인을 식별하세요. 
    자세한 내용을 보려면 [여기](https://aka.ms/vscodeJupyterKernelCrash)를 클릭하세요. 
    자세한 내용은 Jupyter [로그]('command:jupyter.viewOutput')를 참조하세요.

    ```

---

* **`실제 적용 사례`**

*  * **`네이버 지식iN with gemini`**

In [ ]:
# 시뮬레이션: 네이버 지식iN FAQ 시스템 (Gemini 버전)
faq_cache = {
    "휴대폰 요금 문의": "통신사별 요금제는 웹사이트에서 확인 가능합니다.",
    "대학교 입시 정보": "입시 일정은 각 대학 홈페이지를 참고하세요.",
    "코로나 격리 기간": "현재 격리 기간은 7일입니다.",
    # 실제로는 수천 개의 FAQ
}

class NaverKnowledgeSystemWithGemini:
    def __init__(self):
        self.cached_embeddings = cached_embeddings                          # Gemini 캐시백드 임베딩
        self.daily_queries = 0
        self.cache_hits = 0
    
    def answer_question(self, question: str):
        self.daily_queries += 1
        
        # 캐시 확인 (실제로는 벡터 유사도로 매칭)
        for faq_q, faq_a in faq_cache.items():
            if question in faq_q or faq_q in question:
                self.cache_hits += 1
                return f"💡 FAQ 답변: {faq_a}"
        
        # 새로운 질문인 경우 Gemini AI 답변 생성
        return f"🤖 Gemini 답변: {question}에 대한 맞춤형 답변입니다."
    
    def get_stats(self):
        hit_rate = (self.cache_hits / self.daily_queries) * 100 if self.daily_queries > 0 else 0
        return f"일일 질문: {self.daily_queries}, 캐시 적중률: {hit_rate:.1f}%"

# 시뮬레이션
naver_system = NaverKnowledgeSystemWithGemini()

questions = [
    "휴대폰 요금이 궁금해요",
    "대학교 어떻게 들어가나요?", 
    "코로나 걸렸는데 언제까지 격리해야 하나요?",
    "휴대폰 요금제 추천해주세요",  # 유사한 질문
]

for q in questions:
    answer = naver_system.answer_question(q)
    print(f"Q: {q}")
    print(f"A: {answer}\n")

# 결과 출력
print("📊", naver_system.get_stats())

*  * **`개인 예시`**

In [ ]:
class JayAIEducationPlatformWithGemini:
    def __init__(self):
        self.course_materials = {
            "프롬프트 엔지니어링": "효과적인 AI 대화 기술과 실무 적용법",
            "LangChain 실습": "RAG 시스템 구축부터 배포까지",
            "캐시백드 임베딩": "AI 성능 최적화의 핵심 기술",
            "Microsoft 365 Copilot": "업무 자동화와 생산성 향상"
        }
        
        self.cached_embeddings = cached_embeddings  # Gemini 캐시백드 임베딩
        self.monthly_api_cost = 0
        
    def answer_student_question(self, question: str, course: str):
        """수강생 질문에 실시간 답변 (Gemini 버전)"""
        
        # 캐시 확인으로 즉시 답변 (0.05초)
        if course in self.course_materials:
            context = self.course_materials[course]
            return f"📚 {course} 관련: {context}을 참고하여 {question}에 답변드립니다."
        
        # 새로운 내용은 Gemini AI 생성 (1초 - 더 빠름)
        self.monthly_api_cost += 0.005                                  # Gemini는 더 저렴
        return f"🤖 Gemini 답변: {question}에 대한 맞춤형 설명입니다."
    
    def generate_personalized_content(self, student_level: str, topic: str):
        """개인별 맞춤 학습 자료 생성 (Gemini 버전)"""
        cache_key = f"{student_level}_{topic}"
        
        # 캐시된 개인화 콘텐츠 확인
        personalized_content = f"{student_level} 수준의 {topic} 학습자료"
        return f"✨ Gemini 맞춤 자료: {personalized_content}"

# Jay의 플랫폼 시뮬레이션 (Gemini 버전)
jay_platform = JayAIEducationPlatformWithGemini()

# 수강생 질문들
student_questions = [
    ("프롬프트를 어떻게 작성하나요?", "프롬프트 엔지니어링"),
    ("RAG 시스템 구축이 어려워요", "LangChain 실습"),
    ("캐시 적용이 잘 안돼요", "캐시백드 임베딩"),
    ("프롬프트 최적화 방법이 궁금해요", "프롬프트 엔지니어링"),                     # 유사 질문
]

print("🎓 Jay의 AI 교육 플랫폼 - Gemini 실시간 Q&A")
print("=" * 50)

for question, course in student_questions:
    answer = jay_platform.answer_student_question(question, course)
    print(f"👨‍🎓 학생: {question}")
    print(f"👨‍🏫 Jay: {answer}\n")

# 출력
print(f"💰 월 API 비용: ${jay_platform.monthly_api_cost:.3f}")
print("📈 Gemini 예상 효과:")
print("   - 수강생 만족도 45% 향상")  
print("   - 실시간 답변으로 학습 효율성 증대")
print("   - API 비용 95% 절약 (Gemini가 더 저렴)")

In [ ]:
# 새로운 파일 저장소와 키 함수
from langchain.storage import LocalFileStore

file_store = LocalFileStore(cache_dir)

def fixed_safe_key_encoder(text: str) -> str:
    """파일시스템 안전 키 생성 (콜론 제거)"""
    hash_object = hashlib.sha256(text.encode('utf-8'))
    hex_dig = hash_object.hexdigest()
    return f"gemini_embedding_001_{hex_dig}"    # 언더스코어 사용

# 캐시백드 임베딩 재생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=base_embeddings,
    document_embedding_cache=file_store,
    key_encoder=fixed_safe_key_encoder          # 수정된 함수 사용
)

print("✅ 파일시스템 안전 캐시백드 임베딩 준비 완료!")

✅ 파일시스템 안전 캐시백드 임베딩 준비 완료!


In [ ]:
import hashlib

def safe_key_encoder(text: str) -> str:
    """
    텍스트를 안전한 캐시 키로 변환 (수정된 버전)
    - 콜론(:) 대신 언더스코어(_) 사용
    - 파일시스템에서 안전한 문자만 사용
    """
    hash_object = hashlib.sha256(text.encode('utf-8'))
    hex_dig = hash_object.hexdigest()
    return f"gemini_embedding_001_{hex_dig}"        # 콜론을 언더스코어로 변경!

# 캐시백드 임베딩 다시 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=base_embeddings,  
    document_embedding_cache=file_store,    
    key_encoder=safe_key_encoder                    # 수정된 키 생성 함수
)

print("✅ 수정된 Gemini 캐시백드 임베딩 생성 완료!")        # ✅ 수정된 Gemini 캐시백드 임베딩 생성 완료!

---

* **`Task Type별 캐시 전략`**

In [ ]:
class GeminiTaskSpecificCache:
    """Task Type별 특화 캐시 시스템"""
    
    def __init__(self):
        # Task Type별 별도 캐시
        self.document_cache = LocalFileStore("../08_Embedding/cache/documents/")
        self.query_cache = LocalFileStore("../08_Embedding/cache/queries/")
        self.similarity_cache = LocalFileStore("../08_Embedding/cache/similarity/")
        
    def create_task_specific_embeddings(self, task_type: str):
        """Task Type별 캐시백드 임베딩 생성"""
        
        # Task Type별 임베딩 모델
        embeddings = GoogleGenerativeAIEmbeddings(
            #model="gemini-embedding-001",
            model="models/gemini-embedding-001",
            task_type=task_type,
            google_api_key=os.getenv("GOOGLE_API_KEY")
        )
        
        # Task Type별 캐시 선택
        cache_mapping = {
            "retrieval_document": self.document_cache,
            "retrieval_query": self.query_cache,
            "semantic_similarity": self.similarity_cache
        }
        
        cache_store = cache_mapping.get(task_type, self.document_cache)
        
        # 캐시백드 임베딩 생성
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings=embeddings,
            document_embedding_cache=cache_store,
            key_encoder=lambda x: f"{task_type}:{hashlib.sha256(x.encode()).hexdigest()}"
        )
        
        return cached_embeddings
    
    def get_optimal_embedding(self, text: str, purpose: str):
        """목적에 따른 최적 임베딩 선택"""
        
        task_mapping = {
            "search": "retrieval_query",
            "index": "retrieval_document", 
            "compare": "semantic_similarity"
        }
        
        task_type = task_mapping.get(purpose, "retrieval_document")
        embeddings = self.create_task_specific_embeddings(task_type)
        
        return embeddings.embed_query(text)

# 사용 예시
gemini_cache = GeminiTaskSpecificCache()

# 문서 인덱싱용
doc_embedding = gemini_cache.get_optimal_embedding(
    "Jay의 AI 교육 자료", 
    purpose="index"
)

# 검색 쿼리용
query_embedding = gemini_cache.get_optimal_embedding(
    "AI 교육 과정을 찾고 있어요", 
    purpose="search"
)
# E0000 00:00:1758627822.821127 1589277 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
# E0000 00:00:1758627823.410137 1589277 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.

* **`gemini` 멀티모달 캐싱**

In [ ]:
class GeminiMultimodalCache:
    """Gemini 멀티모달 캐싱 시스템 (텍스트 + 이미지)"""
    
    def __init__(self):
        self.text_cache = LocalFileStore("../08_Embedding/cache/text/")
        self.multimodal_cache = LocalFileStore("../08_Embedding/cache/multimodal/")
        
    def create_content_hash(self, content: dict) -> str:
        """텍스트 + 이미지 콘텐츠 해시 생성"""
        content_str = ""
        
        if "text" in content:
            content_str += content["text"]
            
        if "image_path" in content:
            # 이미지 파일 해시도 포함
            with open(content["image_path"], "rb") as f:
                image_hash = hashlib.md5(f.read()).hexdigest()
            content_str += f"_img_{image_hash}"
            
        return hashlib.sha256(content_str.encode()).hexdigest()
    
    def cache_multimodal_embedding(self, content: dict):
        """멀티모달 콘텐츠 임베딩 캐싱"""
        content_hash = self.create_content_hash(content)
        cache_key = f"multimodal:{content_hash}"
        
        print(f"📊 멀티모달 콘텐츠 캐싱: {cache_key[:20]}...")
        
        # 실제 구현에서는 Gemini 멀티모달 API 호출
        # embedding = gemini_multimodal_api.embed(content)
        
        return cache_key

# 사용 예시 (미래 기능)
multimodal_cache = GeminiMultimodalCache()

content = {
    "text": "Jay의 AI 교육 과정 소개",
    "image_path":"../08_Embedding/images/embedding_1.png"
    #"image_path": "./images/course_intro.png"
}

cache_key = multimodal_cache.cache_multimodal_embedding(content)
# 📊 멀티모달 콘텐츠 캐싱: multimodal:8d549e539...

* **`gemini` 최적화 팁**

In [ ]:
def optimize_gemini_caching():
    """Gemini 캐싱 최적화 전략"""
    
    optimization_tips = {
        "1. Task Type 활용": {
            "설명": "용도에 맞는 task_type 명시",
            "예시": "검색용은 retrieval_query, 인덱싱용은 retrieval_document",
            "효과": "정확도 10-15% 향상"
        },
        
        "2. 배치 처리": {
            "설명": "여러 텍스트를 한 번에 처리",
            "예시": "embed_documents() 사용으로 API 호출 최소화",
            "효과": "처리 시간 40% 단축"
        },
        
        "3. 캐시 키 최적화": {
            "설명": "task_type을 포함한 키 생성",
            "예시": "retrieval_document:abc123 vs retrieval_query:abc123",
            "효과": "캐시 충돌 방지"
        },
        
        "4. 오류 재시도": {
            "설명": "Gemini API 호출 실패시 재시도 로직",
            "예시": "exponential backoff 적용",
            "효과": "안정성 95% 이상 확보"
        }
    }
    
    print("🎯 Gemini 캐싱 최적화 가이드:")
    for tip, details in optimization_tips.items():
        print(f"\n{tip}:")
        print(f"   설명: {details['설명']}")
        print(f"   예시: {details['예시']}")
        print(f"   효과: {details['효과']}")

optimize_gemini_caching()

<small>

* 셀 출력

    ```markdown
    🎯 Gemini 캐싱 최적화 가이드:

    1. Task Type 활용:
    설명: 용도에 맞는 task_type 명시
    예시: 검색용은 retrieval_query, 인덱싱용은 retrieval_document
    효과: 정확도 10-15% 향상

    2. 배치 처리:
    설명: 여러 텍스트를 한 번에 처리
    예시: embed_documents() 사용으로 API 호출 최소화
    효과: 처리 시간 40% 단축

    3. 캐시 키 최적화:
    설명: task_type을 포함한 키 생성
    예시: retrieval_document:abc123 vs retrieval_query:abc123
    효과: 캐시 충돌 방지

    4. 오류 재시도:
    설명: Gemini API 호출 실패시 재시도 로직
    예시: exponential backoff 적용
    효과: 안정성 95% 이상 확보
    ```

* action-plan

In [ ]:
jay_gemini_plan = {
    "1주차": {
        "목표": "Gemini 캐시백드 임베딩 마스터",
        "작업": [
            "Google AI Studio API 키 발급",
            "Task Type별 임베딩 테스트",
            "기존 OpenAI 코드를 Gemini로 마이그레이션"
        ],
        "예상효과": "API 비용 60% 절약"
    },
    
    "2주차": {
        "목표": "교육 플랫폼에 Gemini 적용",
        "작업": [
            "강의 자료를 task_type별로 최적화",
            "실시간 Q&A에 캐시백드 Gemini 적용",
            "성능 모니터링 시스템 구축"
        ],
        "예상효과": "응답 속도 80% 향상"
    },
    
    "3주차": {
        "목표": "고급 최적화 및 확장",
        "작업": [
            "멀티 task_type 캐시 시스템",
            "한국어 특화 최적화",
            "비용 대시보드 구축"
        ],
        "예상효과": "전체 비용 95% 절약"
    },
    
    "4주차": {
        "목표": "Gemini 전문 강의 콘텐츠 제작",
        "작업": [
            "'Gemini vs OpenAI 비교' 강의 제작",
            "실무 사례 정리 및 배포",
            "수강생 대상 Gemini 워크샵"
        ],
        "예상효과": "차별화된 강의로 수강생 유치"
    }
}

print("🎯 Jay의 Gemini 마스터 플랜")
print("=" * 50)

for week, plan in jay_gemini_plan.items():
    print(f"\n📅 {week}:")
    print(f"   목표: {plan['목표']}")
    print(f"   주요 작업:")
    for task in plan['작업']:
        print(f"      • {task}")
    print(f"   예상 효과: {plan['예상효과']}")

<small>

* 셀 출력

    ```markdown
    🎯 Jay의 Gemini 마스터 플랜
    ==================================================

    📅 1주차:
    목표: Gemini 캐시백드 임베딩 마스터
    주요 작업:
        • Google AI Studio API 키 발급
        • Task Type별 임베딩 테스트
        • 기존 OpenAI 코드를 Gemini로 마이그레이션
    예상 효과: API 비용 60% 절약

    📅 2주차:
    목표: 교육 플랫폼에 Gemini 적용
    주요 작업:
        • 강의 자료를 task_type별로 최적화
        • 실시간 Q&A에 캐시백드 Gemini 적용
        • 성능 모니터링 시스템 구축
    예상 효과: 응답 속도 80% 향상

    📅 3주차:
    목표: 고급 최적화 및 확장
    주요 작업:
        • 멀티 task_type 캐시 시스템
        • 한국어 특화 최적화
        • 비용 대시보드 구축
    예상 효과: 전체 비용 95% 절약

    📅 4주차:
    목표: Gemini 전문 강의 콘텐츠 제작
    주요 작업:
        • 'Gemini vs OpenAI 비교' 강의 제작
        • 실무 사례 정리 및 배포
        • 수강생 대상 Gemini 워크샵
    예상 효과: 차별화된 강의로 수강생 유치
    ```

* 마무리

In [ ]:
print("""
🚀 Jay's Gemini Cache Success Formula:

1️⃣ Gemini 임베딩 모델 적용 (60% 비용 절약)
2️⃣ 캐시백드 시스템 구축 (95% 속도 향상)  
3️⃣ Task Type 최적화 (15% 정확도 향상)
4️⃣ 실무 프로젝트 적용 (경험 축적)
5️⃣ 교육 콘텐츠화 (수익 창출)

💡 결과: 비용 ↓95%, 속도 ↑200배, 만족도 ↑50%

🎓 Jay, 이제 Gemini + 캐시백드 임베딩의 
    진정한 마스터가 될 준비 완료! 🔥
""")

<small>

* 셀 출력

    ```markdown
    🚀 Jay's Gemini Cache Success Formula:

    1️⃣ Gemini 임베딩 모델 적용 (60% 비용 절약)
    2️⃣ 캐시백드 시스템 구축 (95% 속도 향상)  
    3️⃣ Task Type 최적화 (15% 정확도 향상)
    4️⃣ 실무 프로젝트 적용 (경험 축적)
    5️⃣ 교육 콘텐츠화 (수익 창출)

    💡 결과: 비용 ↓95%, 속도 ↑200배, 만족도 ↑50%

    🎓 Jay, 이제 Gemini + 캐시백드 임베딩의 
        진정한 마스터가 될 준비 완료! 🔥
    ```

---

* *next: **`허깅페이스 임베딩 (HuggingFace Embeddings)`***

---